In [3]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-apple-darwin
info: syncing channel updates for 'stable-x86_64-apple-darwin'
info: latest update on 2024-10-17, rust version 1.82.0 (f6e511eec 2024-10-15)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
 59.6 MiB /  59.6 MiB (100 %)  33.0 MiB/s in  1s ETA:  0s
info: downloading component 'rustfmt'
info: installing component 'cargo'
  7.1 MiB /   7.1 MiB (100 %)   4.9 MiB/s in  1s ETA:  0s
info: installing component 'clippy'
info: installing component 'rust-docs'
 16.3 MiB /  16.3 MiB (100 %)   1.9 MiB/s in 14s ETA:  0s    
info: installing component 'rust-std'
 24.9 MiB /  24.9 MiB (100 %)   8.6 MiB/s in  2s ETA:  0s
info: installing component 'rustc'
 59.6 MiB /  59.6 MiB (100 %)   9.3 MiB/s in  6s ETA:  0s
info: installing component 'rustfmt'
info: defau

In [5]:
import os
os.environ['PATH'] += os.pathsep + os.path.expanduser('~/.cargo/bin')

In [7]:
!rustc --version
!cargo --version


rustc 1.82.0 (f6e511eec 2024-10-15)
cargo 1.82.0 (8f40fc59f 2024-08-21)


In [9]:
!pip install "quri-parts[braket,cirq,qiskit,qulacs,tket]"


  Using cached quri_parts-0.20.3-py3-none-any.whl.metadata (6.6 kB)
  Using cached quri_parts_algo-0.20.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached quri_parts_braket-0.20.3-py3-none-any.whl.metadata (1.4 kB)
  Using cached quri_parts_chem-0.20.3-py3-none-any.whl.metadata (1.1 kB)
  Using cached quri_parts_circuit-0.20.3-py3-none-any.whl.metadata (1.1 kB)
  Using cached quri_parts_cirq-0.20.3-py3-none-any.whl.metadata (1.1 kB)
  Using cached quri_parts_core-0.20.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached quri_parts_qiskit-0.20.3-py3-none-any.whl.metadata (1.3 kB)
  Using cached quri_parts_qulacs-0.20.3-py3-none-any.whl.metadata (1.3 kB)
  Using cached quri_parts_tket-0.20.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached amazon_braket_schemas-1.22.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached amazon_braket_sdk-1.88.1-py3-none-any.whl.metadata (14 kB)
  Using cached quri_parts_rust-0.20.3.tar.gz (629 kB)
  Installing build dependencies ... done
  Getting requirem

In [13]:
import sys
print("Pythonのバージョン:", sys.version)


Pythonのバージョン: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 10:14:12) [Clang 14.0.6 ]


In [5]:
from quri_parts.algo.ansatz import build_entangler_map, EntanglementPatternType, TwoLocal

def add_rotation_gate(circuit, arg):
    layer_index, qubit_index = arg
    theta = circuit.add_parameter("theta")
    if layer_index % 2 == 0:
        circuit.add_ParametricRY_gate(qubit_index, theta)
    else:
        circuit.add_ParametricRZ_gate(qubit_index, theta)

def add_entanglement_gate(circuit, arg):
    layer_index, (i, j) = arg
    circuit.add_CZ_gate(i, j)

qubit_count = 4
reps = 2
entangler_map_seq = build_entangler_map(
    qubit_count, [EntanglementPatternType.LINEAR] * reps
)

two_local = TwoLocal(
    qubit_count=qubit_count,
    layer_pattern="rerer",
    rot_layer_maker=add_rotation_gate,
    ent_layer_maker=add_entanglement_gate,
    rotation_indices=range(4),
    entangler_map_seq=entangler_map_seq
)


In [27]:
print(two_local)

In [5]:
import numpy as np
from quri_parts.circuit.gates import PauliRotation
from collections.abc import Sequence
from typing import Optional

from quri_parts.circuit import (
    ImmutableLinearMappedParametricQuantumCircuit,
    LinearMappedParametricQuantumCircuit,
)


def Rxx_gate(indx, theta):
    gate = PauliRotation(indx, (1, 1), theta) #X gate: pauli_ids=1
    return gate

def _add_rz_gates(circuit, arg):
    layer_index, qubit_index = arg
    theta = circuit.add_parameter("theta")
    circuit.add_ParametricRZ_gate(qubit_index, theta)
    

def _add_rxx_gates(circuit, arg):
    layer_index, (i, j) = arg
    if layer_index % 2 == 0:
        circuit.add_gate(Rxx_gate((i,j), np.pi/2))
    else:
        circuit.add_gate(Rxx_gate((i,j), -np.pi/2))        

In [17]:
#example
qubit_count = 2
reps = 2
entangler_map_seq = build_entangler_map(
    qubit_count, [EntanglementPatternType.LINEAR] * reps
)

two_local = TwoLocal(
    qubit_count=qubit_count,
    layer_pattern="ere",
    rot_layer_maker=add_rotation_gate,
    ent_layer_maker=add_entanglement_gate,
    rotation_indices=range(qubit_count),
    entangler_map_seq=entangler_map_seq
)


In [97]:
from quri_parts.circuit.utils.circuit_drawer import draw_circuit

draw_circuit(two_local)

   ___     ___     ___  
  |PR |   |PRZ|   |PR | 
--|0  |---|1  |---|3  |-
  |   |   |___|   |   | 
  |   |    ___    |   | 
  |   |   |PRZ|   |   | 
--|   |---|2  |---|   |-
  |___|   |___|   |___| 


In [19]:
class Ansatz(ImmutableLinearMappedParametricQuantumCircuit):
    _add_rotation_gates = _add_rz_gates
    _add_entanglement_gate = _add_rxx_gates
    def __init__(
        self,
        qubit_count: int,
        reps: int,
        rotation_indices: Optional[Sequence[int]] = None,
        entangler_map_seq: Optional[Sequence[Sequence[tuple[int, int]]]] = None,
    ):
        if rotation_indices is None:
            rotation_indices = list(range(qubit_count))
        if entangler_map_seq is None:
            entangler_map_seq = build_entangler_map(
                qubit_count, [EntanglementPatternType.LINEAR] * reps
            )

        circuit = TwoLocal(
            qubit_count,
            layer_pattern="ere" * reps,
            rot_layer_maker=self.__class__._add_rotation_gates,
            ent_layer_maker=self.__class__._add_entanglement_gate,
            rotation_indices=rotation_indices,
            entangler_map_seq=entangler_map_seq,
        )



NameError: name 'ImmutableLinearMappedParametricQuantumCircuit' is not defined